In [1]:
from scipy.io import loadmat
import numpy as np
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

In [2]:
data = loadmat('DataforFit.mat')


In [3]:
y=data['TrainProperty02']
X=data['averagePcScores']

In [4]:
clf = BayesianRidge(compute_score=True)
clf.fit(X, y)


C:\Users\yucel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=True, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

In [5]:
def err(x1,x2):
    error=abs(x2-x1)/x2
#   print (error)
    return error
    

In [6]:
clf.coef_

array([ 1478.34100418, -2290.52133851,   996.54814514,  2319.75615264,
       -1330.09701347,  -474.29326028])

In [7]:
y_hat=clf.predict(X).reshape(16,1)

ols = LinearRegression()
ols.fit(X, y)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
err(ols.predict(X),y).sum()/16

0.024359576747075586

In [9]:
sum(err(y_hat,y)/16)

array([0.02721972])

In [10]:
y.shape

(16, 1)

In [11]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(X)

16

In [12]:
LOOCVerr=[]
LOOCVerrOLS=[]
LOOCVerrGPR=[]


for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train, X_test, y_train, y_test)
    clf = BayesianRidge(compute_score=True)
    clf.fit(X_train[:,0:3], y_train)
    ols = LinearRegression()
    ols.fit(X_train[:,0:3], y_train)
    y_hat=clf.predict(X_test[:,0:3]).reshape(1,1)
    y_hat2=ols.predict(X_test[:,0:3]).reshape(1,1)
    LOOCVerr.append(err(y_hat,y_test))
    LOOCVerrOLS.append(err(y_hat2,y_test))
    noise=0.5
    rbf = ConstantKernel(1.0) * RBF(length_scale=5.0)
    gpr = GaussianProcessRegressor(kernel=rbf, alpha=noise**2)
    gpr.fit(X_train, y_train)
    yhat_3, cov_s = gpr.predict(X, return_cov=True)
    LOOCVerrGPR.append(err(yhat_3,y_test))
    

    











C:\Users\yucel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] TEST: [0]
[[-0.04321941 -0.0122951   0.00091181 -0.00329273  0.00594966 -0.00025994]
 [ 0.00672726 -0.00259795 -0.00779799 -0.00194821  0.0032816  -0.00170879]
 [-0.03176064  0.03313103  0.00754749  0.0045861  -0.00270223  0.00033475]
 [-0.04680588  0.02704906  0.01464697  0.00594077  0.00362692 -0.0006239 ]
 [ 0.00709518 -0.03669919 -0.00533507  0.00840916  0.00047621 -0.00070186]
 [ 0.03438806 -0.00550495 -0.00792708  0.00581713 -0.00118548  0.0050296 ]
 [ 0.03434151  0.00042198 -0.00678504  0.00546724 -0.00376569  0.0011789 ]
 [ 0.07201958  0.01400266  0.00228164 -0.01763053  0.00433997 -0.0007964 ]
 [-0.01077828 -0.00363166 -0.00537903 -0.00124408 -0.00104492 -0.00144516]
 [-0.0580591  -0.03488848  0.00431069 -0.01351934 -0.00684538  0.00218816]
 [ 0.01223359  0.0286289  -0.00496884 -0.00310027 -0.00250456 -0.00196983]
 [ 0.0075376   0.03337434 -0.00363945 -0.00122466 -0.00321444  0.00124981]
 [-0.02499669  0.01809744  0.0009187

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14 15] TEST: [13]
[[-0.01206686 -0.02311451 -0.00917529  0.00302907  0.00281945 -0.00110773]
 [-0.04321941 -0.0122951   0.00091181 -0.00329273  0.00594966 -0.00025994]
 [ 0.00672726 -0.00259795 -0.00779799 -0.00194821  0.0032816  -0.00170879]
 [-0.03176064  0.03313103  0.00754749  0.0045861  -0.00270223  0.00033475]
 [-0.04680588  0.02704906  0.01464697  0.00594077  0.00362692 -0.0006239 ]
 [ 0.00709518 -0.03669919 -0.00533507  0.00840916  0.00047621 -0.00070186]
 [ 0.03438806 -0.00550495 -0.00792708  0.00581713 -0.00118548  0.0050296 ]
 [ 0.03434151  0.00042198 -0.00678504  0.00546724 -0.00376569  0.0011789 ]
 [ 0.07201958  0.01400266  0.00228164 -0.01763053  0.00433997 -0.0007964 ]
 [-0.01077828 -0.00363166 -0.00537903 -0.00124408 -0.00104492 -0.00144516]
 [-0.0580591  -0.03488848  0.00431069 -0.01351934 -0.00684538  0.00218816]
 [ 0.01223359  0.0286289  -0.00496884 -0.00310027 -0.00250456 -0.00196983]
 [ 0.0075376   0.03337434 -0.003639

In [13]:
np.sum(LOOCVerr)/16

0.05538180147001151

In [14]:
np.sum(LOOCVerrOLS)/16

0.059503841280351015

In [15]:
import GPy

In [16]:
rbf = GPy.kern.RBF(input_dim=1, variance=232.886, lengthscale=0.1216)
gpr = GPy.models.GPRegression(X, y, rbf)

# Fix the noise variance to known value 
gpr.Gaussian_noise.variance = noise**2
gpr.Gaussian_noise.variance.fix()

# Run optimization
#gpr.optimize();

# Display optimized parameter values
display(gpr)

 C:\Users\yucel\Anaconda3\lib\site-packages\GPy\core\gp.py:87: UserWarning:Your kernel has a different input dimension 1 then the given X dimension 6. Be very sure this is what you want and you have not forgotten to set the right input dimenion in your kernel


GP_regression.,value,constraints,priors
rbf.variance,232.886,+ve,
rbf.lengthscale,0.1216,+ve,
Gaussian_noise.variance,0.25,+ve fixed,


In [17]:
# Obtain optimized kernel parameters
l = gpr.rbf.lengthscale.values[0]
sigma_f = np.sqrt(gpr.rbf.variance.values[0])

In [18]:
yyyy=gpr.predict(X)

In [19]:
y

array([[641.7575],
       [586.542 ],
       [602.3725],
       [536.0675],
       [511.0605],
       [766.784 ],
       [700.0695],
       [684.925 ],
       [649.955 ],
       [648.257 ],
       [601.227 ],
       [594.326 ],
       [583.0125],
       [574.538 ],
       [862.9575],
       [676.478 ]])

In [20]:
err(yyyy,y)[0,:,0].sum()/16

0.080388977602395

In [21]:
np.shape(yyyy)

(2, 16, 1)

In [22]:
Testdata = loadmat('Testdatainfo.mat')
y_test=Testdata['RealVals']
X_test=Testdata['TestdataAverageScores']

In [23]:
y_hatBLR=clf.predict(X_test[:,0:3]).reshape(-1,1)

In [24]:
err(y_hatBLR,y_test).sum()/3

0.07552176721311434

In [25]:
y_hatOLS=ols.predict(X_test[:,0:3]).reshape(-1,1)

In [26]:
err(y_hatOLS,y_test).sum()/4

0.05413469119438014

In [27]:
y_hatOLS

array([[704.70010503],
       [669.27850774],
       [836.31330083],
       [590.65602065]])

In [28]:
LOOCVerrOLS

[array([[0.04238877]]),
 array([[0.03774544]]),
 array([[0.07934113]]),
 array([[0.03322749]]),
 array([[0.04482264]]),
 array([[0.06484841]]),
 array([[0.0131202]]),
 array([[0.00874821]]),
 array([[0.15284626]]),
 array([[0.0407673]]),
 array([[0.10832575]]),
 array([[0.02413082]]),
 array([[0.03636845]]),
 array([[0.02888789]]),
 array([[0.16254132]]),
 array([[0.07395138]])]

In [29]:
import openpyxl


In [30]:
xfile = openpyxl.load_workbook('ErrorValues.xlsx')


In [37]:
sheet = xfile.get_sheet_by_name('CLD-LR')
for i in range(1,17):
    c1 = sheet.cell(row = i+1, column = 1) 
    c1.value =(float(LOOCVerrOLS[i-1]))
errors=err(y_hatOLS,y_test)
for i in range(1,5):
    c1 = sheet.cell(row = i+1, column = 7)
    c1.value =(float(errors[i-1]))
xfile.save('ErrorValues.xlsx')

In [38]:
np.sum(errors)/4

0.05413469119438014

In [ ]:
errors

In [32]:
# For BLR
sheet = xfile.get_sheet_by_name('CLD-BLR ')
for i in range(1,17):
    c1 = sheet.cell(row = i+1, column = 1) 
    c1.value =(float(LOOCVerr[i-1]))
errors=err(y_hatBLR,y_test)
for i in range(1,5):
    c1 = sheet.cell(row = i+1, column = 7)
    c1.value =(float(errors[i-1]))
xfile.save('ErrorValues.xlsx')

In [33]:
X_train.shape

(15, 6)

In [34]:
X_train[:,0:3]

array([[-0.01206686, -0.02311451, -0.00917529],
       [-0.04321941, -0.0122951 ,  0.00091181],
       [ 0.00672726, -0.00259795, -0.00779799],
       [-0.03176064,  0.03313103,  0.00754749],
       [-0.04680588,  0.02704906,  0.01464697],
       [ 0.00709518, -0.03669919, -0.00533507],
       [ 0.03438806, -0.00550495, -0.00792708],
       [ 0.03434151,  0.00042198, -0.00678504],
       [ 0.07201958,  0.01400266,  0.00228164],
       [-0.01077828, -0.00363166, -0.00537903],
       [-0.0580591 , -0.03488848,  0.00431069],
       [ 0.01223359,  0.0286289 , -0.00496884],
       [ 0.0075376 ,  0.03337434, -0.00363945],
       [-0.02499669,  0.01809744,  0.00091871],
       [ 0.06441782, -0.03068797,  0.02369518]])